In [ ]:
import numpy as np
import ng_link
from ng_link import NgState, link_utils

In [ ]:
def _zyx_vector_to_5x6(zyx_vector: np.ndarray):
    output = np.zeros(5, 6)
    output[2, 5] = zyx_vector[0]
    output[3, 5] = zyx_vector[1]
    output[4, 5] = zyx_vector[0]
    return output

# Simply applies the same registration across channels, b/c coreg is extra effort for little gain
def create_ng_link(tile_paths: list[list[str]], 
                   tile_layout: np.ndarray, 
                   cx: np.ndarray, 
                   cy: np.ndarray,
                   max_dr: int = 200,
                   opacity: float = 1.0,
                   blend: str = "default",
                   output_json_path: str = ".") -> None:
    # Determine tile translations
    tile_translations_zyx = np.zeros(tile_layout.shape)
    for xi in range(tile_layout.shape[0]):
        for yi in range(tile_layout.shape[1]):
            if xi == 0 and yi == 0:  # TL corner tile is fixed
                continue
            
            if xi == 0: # Tile on top border 
                h_shift = cx[:, 1, xi, yi]
                tile_translations_zyx[xi, yi] = h_shift + tile_translations_zyx[xi - 1, yi]

            elif yi == 0: # Tile on left border
                v_shift = cy[:, 1, xi, yi]
                tile_translations_zyx[xi, yi] = v_shift + tile_translations_zyx[xi, yi - 1]

            else:
                h_shift = cx[:, 1, xi, yi]
                v_shift = cy[:, 1, xi, yi]
                tile_translations_zyx[xi, yi] = v_shift + tile_translations_zyx[xi - 1, yi - 1]

    # Determine voxel size
    vox_sizes = "TODO: Parse from tile_path -> zarray"

    # Determine color
    channel: int = link_utils.extract_channel_from_tile_path(tile_paths[0][0])
    hex_val: int = link_utils.wavelength_to_hex(channel)
    hex_str = f"#{str(hex(hex_val))[2:]}"
    
    # Generate input config
    layers = []  # Nueroglancer Tabs
    input_config = {
        "dimensions": {
            "x": {"voxel_size": vox_sizes[0], "unit": "microns"},
            "y": {"voxel_size": vox_sizes[1], "unit": "microns"},
            "z": {"voxel_size": vox_sizes[2], "unit": "microns"},
            "c'": {"voxel_size": 1, "unit": ""},
            "t": {"voxel_size": 0.001, "unit": "seconds"},
        },
        "layers": layers,
        "showScaleBar": False,
        "showAxisLines": False,
    }

    for channel_tile_paths in tile_paths:
        sources = []  # Tiles within tabs
        layers.append(
            {
                "type": "image",  # Optional
                "source": sources,
                "channel": 0,  # Optional
                "shaderControls": {
                    "normalized": {"range": [0, max_dr]}
                },  # Optional  # Exaspim has low HDR
                "shader": {
                    "color": hex_str,
                    "emitter": "RGB",
                    "vec": "vec3",
                },
                "visible": True,  # Optional
                "opacity": opacity,
                "name": f"CH_{channel}",
                "blend": blend,
            }
        )

        for tile_id, tr_zyx in zip(tile_layout, tile_translations_zyx):
            url = f"s3://aind-open-data/{channel_tile_paths[tile_id]}"
            sources.append(
                {"url": url, "transform_matrix": _zyx_vector_to_5x6(tr_zyx)}
            )
    
    # Generate the link
    neuroglancer_link = NgState(
        input_config=input_config,
        mount_service="s3",
        bucket_path="aind-open-data",
        output_json=output_json_path,
    )
    neuroglancer_link.save_state_as_json()
    print(neuroglancer_link.get_url_link())

    return input_config

In [ ]:
def create_raw_ng_link(tile_paths: list[list[str]], 
                       tile_layout: np.ndarray,
                       max_dr: int = 200,
                       opacity: float = 1.0,
                       blend: str = "default",
                       output_json_path: str = "."):
    cx = cy = np.zeros((3, 1, tile_layout.shape[0], tile_layout.shape[1]))
    create_ng_link(tile_paths, 
                   tile_layout,
                   cx, 
                   cy,
                   max_dr,
                   opacity,
                   blend,
                   output_json_path)

In [ ]:
# Everything required for registration is here, 
# cannot wrap into a function yet due to tile layout. 

import numpy as np
import zarr_io
import coarse_registration

bucket = 'aind-open-data'
downsampling_exp = 2
tile_volumes = []  # Filled with x in outer loop and y in inner loop
for x in range(5):
    for y in range(3):
        dataset = 'exaSPIM_615296_2022-09-28_11-47-06/exaSPIM/'
        path = f'tile_x_000{x}_y_000{y}_z_0000_ch_488/{downsampling_exp}'
        tile = zarr_io.open_zarr_s3(bucket, dataset + path)
        tile_volumes.append(tile.T[:,:,:,0,0])

# Inferred from Neuroglancer
tile_layout = np.array([[14, 11, 8, 5, 2], 
                        [13, 10, 7, 4, 1], 
                        [12,  9, 6, 3, 0]])
cx, cy = coarse_registration.compute_coarse_offsets(tile_layout, tile_volumes)


# ^ Very close to 'testing' script
# Just need to format the tile_path argument a little. 

In [ ]:
# What's left: 
# - Testing script, move the code into repo generated by aind-capsule-template
# - JAX capsule environment
# - Capsule Inputs (not technically necessary right now)

# (That should be enough work for tomorrow)